In [27]:
import pandas as pd
import numpy as np
import plotly.express as px

from openpyxl import Workbook, load_workbook
from xlsxwriter import Workbook

import requests
from datetime import datetime
from pytz import timezone
import pytz


import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from datetime import datetime
from pytz import timezone
import pytz
# Importando a Classe Cores
class bcolors:
    OKGREEN = '\033[92m' #GREEN
    OkBLUE = '\033[94m' #BLUE
    WARNING = '\033[93m' #YELLOW
    FAIL = '\033[91m' #RED
    RESET = '\033[0m' #RESET COLOR


In [28]:
df = pd.read_excel("C:/Users/thiag/OneDrive/curso python/MODELOS LEANDRO/Base_FlahScore.xlsx")

df = df[['Date','Time','League','Season','Round','Home','Away','HT_Goals_H','HT_Goals_A','FT_Goals_H','FT_Goals_A','Odd_H',
'Odd_D','Odd_A','Odd_Over25','Odd_Under25','Odd_BTTS_Yes','Odd_BTTS_No']]

df.columns = ['Date', 'Time', 'League', 'Season', 'Round', 'Home', 'Away', 'HT_Goals_H', 'HT_Goals_A', 
                'FT_Goals_H', 'FT_Goals_A', 'FT_Odd_H', 'FT_Odd_D', 'FT_Odd_A', 'FT_Odd_Over25', 'FT_Odd_Under25', 
                'FT_Odd_BTTS_Yes','FT_Odd_BTTS_No']

In [29]:
df["Total_Goals"] = df["FT_Goals_H"] + df["FT_Goals_A"]

df['HT_Total_Goals'] = df['HT_Goals_H'] + df['HT_Goals_A']

df['over05ft'] = df.apply(lambda row: 1 if (row['Total_Goals'] >= 1) else 0, axis=1)

df['over05ht'] = df.apply(lambda row: 1 if (row['HT_Total_Goals'] >= 1) else 0, axis=1)

df['over15ft'] = df.apply(lambda row: 1 if (row['Total_Goals'] >= 2) else 0, axis=1)

df['over25ft'] = df.apply(lambda row: 1 if (row['Total_Goals'] >= 3) else 0, axis=1)

df['over35ft'] = df.apply(lambda row: 1 if (row['Total_Goals'] >= 4) else 0, axis=1)

df['over45ft'] = df.apply(lambda row: 1 if (row['Total_Goals'] >= 5) else 0, axis=1) 

df['btts_ft'] = df.apply(lambda row: 1 if (row['FT_Goals_H'] > 0 and row['FT_Goals_A'] > 0 ) else 0, axis=1)

print('pronto')

pronto


In [30]:
#


#flt = ((df.FT_Odd_H != 0) & (df.FT_Odd_D != 0) & (df.FT_Odd_A != 0))
#df = df[flt]


df.loc[df['FT_Goals_H'] > df['FT_Goals_A'], 'FT_Result'] = "H"
df.loc[df['FT_Goals_H'] == df['FT_Goals_A'], 'FT_Result'] = "D"
df.loc[df['FT_Goals_H'] < df['FT_Goals_A'], 'FT_Result'] = "A"


df.loc[df['Total_Goals'] > 1, 'FT_Result_1.5_ft'] = "S"
df.loc[df['Total_Goals'] <= 1, 'FT_Result_1.5_ft'] = "N"

df.loc[df['Total_Goals'] > 2, 'FT_Result_2.5_ft'] = "S"
df.loc[df['Total_Goals'] <= 2, 'FT_Result_2.5_ft'] = "N"

df.loc[df['HT_Total_Goals'] > 0, 'FT_Result_0.5_ht'] = "S"
df.loc[df['HT_Total_Goals'] == 0 , 'FT_Result_0.5_ht'] = "N"

df.loc[df['Total_Goals'] > 0, 'FT_Result_0.5_ft'] = "S"
df.loc[df['Total_Goals'] == 0, 'FT_Result_0.5_ft'] = "N"

df.loc[(df['FT_Goals_H'] >= df['FT_Goals_A']), 'LayAway'] = 1
df.loc[(df['FT_Goals_H'] < df['FT_Goals_A']), 'LayAway'] = 0

df.loc[(df['FT_Goals_H'] <= df['FT_Goals_A']), 'LayHome'] = 1
df.loc[(df['FT_Goals_H'] > df['FT_Goals_A']), 'LayHome'] = 0

df.loc[(df['FT_Goals_H'] != df['FT_Goals_A']), 'LayDraw'] = 1
df.loc[(df['FT_Goals_H'] == df['FT_Goals_A']), 'LayDraw'] = 0

df.loc[(df['FT_Goals_H'] < df['FT_Goals_A']), 'BackAway'] = 1
df.loc[(df['FT_Goals_H'] >= df['FT_Goals_A']), 'BackAway'] = 0

df.loc[(df['FT_Goals_H'] > df['FT_Goals_A']), 'BackHome'] = 1
df.loc[(df['FT_Goals_H'] <= df['FT_Goals_A']), 'BackHome'] = 0

df.loc[(df['FT_Goals_H'] == df['FT_Goals_A']), 'BackDraw'] = 1
df.loc[(df['FT_Goals_H'] != df['FT_Goals_A']), 'BackDraw'] = 0

# Profit - LAY
green =   1 / ((df.FT_Odd_H) - 0.85)
red = -1

df.loc[df['LayHome'] == 1, 'Profit_lay_home']  = green
df.loc[df['LayHome'] == 0, 'Profit_lay_home']  = red


green =   1 / ((df.FT_Odd_A) - 0.85)
red = -1

df.loc[df['LayAway'] == 1, 'Profit_lay_away']  = green
df.loc[df['LayAway'] == 0, 'Profit_lay_away']  = red

green =   1 / ((df.FT_Odd_D) - 0.85)
red = -1

df.loc[df['LayDraw'] == 1, 'Profit_lay_draw']  = green
df.loc[df['LayDraw'] == 0, 'Profit_lay_draw']  = red


# Profit
green = df.FT_Odd_D - 1
red = -1

df.loc[df['FT_Result'] == "D", 'Profit_back_draw'] = green
df.loc[df['FT_Result'] != "D", 'Profit_back_draw'] = red

green = df.FT_Odd_H - 1
red = -1

df.loc[df['FT_Result'] == "H", 'Profit_back_home'] = green
df.loc[df['FT_Result'] != "H", 'Profit_back_home'] = red


green = df.FT_Odd_A - 1
red = -1

df.loc[df['FT_Result'] == "A", 'Profit_back_away'] = green
df.loc[df['FT_Result'] != "A", 'Profit_back_away'] = red

green = df.FT_Odd_Over25 - 1
red = -1

df.loc[df['FT_Result_2.5_ft'] == "S", 'Profit_25_ft'] = green
df.loc[df['FT_Result_2.5_ft'] != "S", 'Profit_25_ft'] = red

# Probabilidades
df['p_H'] = round((1 / df.FT_Odd_H), 3)
df['p_D'] = round((1 / df.FT_Odd_D), 3)
df['p_A'] = round((1 / df.FT_Odd_A), 3)
df['p_Over'] = round((1 / df.FT_Odd_Over25), 3)

# CV das Odds do Match Odds
df['CV_HDA'] = round((df[['p_H','p_D','p_A']].std(ddof=0, axis=1) / df[['p_H','p_D','p_A']].mean(axis=1)), 3)

# Pontos
df.loc[df['FT_Goals_H'] > df['FT_Goals_A'], 'Ptos_H'] = 3
df.loc[df['FT_Goals_H'] == df['FT_Goals_A'], 'Ptos_H'] = 1
df.loc[df['FT_Goals_H'] < df['FT_Goals_A'], 'Ptos_H'] = 0

df.loc[df['FT_Goals_H'] < df['FT_Goals_A'], 'Ptos_A'] = 3
df.loc[df['FT_Goals_H'] == df['FT_Goals_A'], 'Ptos_A'] = 1
df.loc[df['FT_Goals_H'] > df['FT_Goals_A'], 'Ptos_A'] = 0

df["Ptos_H"] = df["Ptos_H"].astype('int64')
df["Ptos_A"] = df["Ptos_A"].astype('int64')

df['Media_Ptos_H'] = df.groupby('Home')['Ptos_H'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True)
df['Media_Ptos_A'] = df.groupby('Away')['Ptos_A'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True)

df['Media_Ptos_H'] = df.groupby('Home')['Media_Ptos_H'].shift(1)
df['Media_Ptos_A'] = df.groupby('Away')['Media_Ptos_A'].shift(1)

df['DesvPad_Ptos_H'] = df.groupby('Home')['Ptos_H'].rolling(window=10, min_periods=2).std(ddof=0).reset_index(0,drop=True)
df['DesvPad_Ptos_A'] = df.groupby('Away')['Ptos_A'].rolling(window=10, min_periods=2).std(ddof=0).reset_index(0,drop=True)

df['DesvPad_Ptos_H'] = df.groupby('Home')['DesvPad_Ptos_H'].shift(1)
df['DesvPad_Ptos_A'] = df.groupby('Away')['DesvPad_Ptos_A'].shift(1)

df['CV_Ptos_H'] = df['DesvPad_Ptos_H'] / df['Media_Ptos_H']
df['CV_Ptos_A'] = df['DesvPad_Ptos_A'] / df['Media_Ptos_A']

# Valor do Gol
df['VG_H'] = df.FT_Goals_H * df.p_A
df['VG_A'] = df.FT_Goals_A * df.p_H

df['Media_VG_H'] = df.groupby('Home')['VG_H'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True)
df['Media_VG_A'] = df.groupby('Away')['VG_A'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True)

df['Media_VG_H'] = df.groupby('Home')['Media_VG_H'].shift(1)
df['Media_VG_A'] = df.groupby('Away')['Media_VG_A'].shift(1)


df['DesvPad_VG_H'] = df.groupby('Home')['VG_H'].rolling(window=10, min_periods=2).std(ddof=0).reset_index(0,drop=True)
df['DesvPad_VG_A'] = df.groupby('Away')['VG_A'].rolling(window=10, min_periods=2).std(ddof=0).reset_index(0,drop=True)

df['DesvPad_VG_H'] = df.groupby('Home')['DesvPad_VG_H'].shift(1)
df['DesvPad_VG_A'] = df.groupby('Away')['DesvPad_VG_A'].shift(1)

df['CV_VG_H'] = round((df['DesvPad_VG_H'] / df['Media_VG_H']), 3)
df['CV_VG_A'] = round((df['DesvPad_VG_A'] / df['Media_VG_A']), 3)

# Custo do Gol
df['CG_H'] = df.p_H / df.FT_Goals_H
df['CG_A'] = df.p_A / df.FT_Goals_A

df['Media_CG_H'] = df.groupby('Home')['CG_H'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True)
df['Media_CG_A'] = df.groupby('Away')['CG_A'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True)

df['Media_CG_H'] = df.groupby('Home')['Media_CG_H'].shift(1)
df['Media_CG_A'] = df.groupby('Away')['Media_CG_A'].shift(1)

df['DesvPad_CG_H'] = df.groupby('Home')['CG_H'].rolling(window=10, min_periods=2).std(ddof=0).reset_index(0,drop=True)
df['DesvPad_CG_A'] = df.groupby('Away')['CG_A'].rolling(window=10, min_periods=2).std(ddof=0).reset_index(0,drop=True)

df['DesvPad_CG_H'] = df.groupby('Home')['DesvPad_CG_H'].shift(1)
df['DesvPad_CG_A'] = df.groupby('Away')['DesvPad_CG_A'].shift(1)

df['CV_CG_H'] = round((df['DesvPad_CG_H'] / df['Media_CG_H']), 3)
df['CV_CG_A'] = round((df['DesvPad_CG_A'] / df['Media_CG_A']), 3)


#df.replace(np.inf, 1, inplace=True)
#df = drop_reset_index(df)
print('pronto')

pronto


In [31]:
# media de gols 5 jogos

df['mean55_gols_feitos_home_ft'] = df.groupby('Home')['FT_Goals_H'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True)
df['mean55_gols_sofridos_home_ft'] = df.groupby('Home')['FT_Goals_A'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True)

df['mean55_gols_feitos_home_ft'] = df.groupby('Home')['mean55_gols_feitos_home_ft'].shift(1)
df['mean55_gols_sofridos_home_ft'] = df.groupby('Home')['mean55_gols_sofridos_home_ft'].shift(1)

df['mean55_gols_feitos_away_ft'] = df.groupby('Away')['FT_Goals_A'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True)
df['mean55_gols_sofridos_away_ft'] = df.groupby('Away')['FT_Goals_H'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True)

df['mean55_gols_feitos_away_ft'] = df.groupby('Away')['mean55_gols_feitos_away_ft'].shift(1)
df['mean55_gols_sofridos_away_ft'] = df.groupby('Away')['mean55_gols_sofridos_away_ft'].shift(1)

# desvio padrao 5 jogos

df['std5_gols_feitos_home_ft'] = df.groupby('Home')['FT_Goals_H'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True)
df['std5_gols_sofridos_home_ft'] = df.groupby('Home')['FT_Goals_A'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True)

df['std5_gols_feitos_home_ft'] = df.groupby('Home')['std5_gols_feitos_home_ft'].shift(1)
df['std5_gols_sofridos_home_ft'] = df.groupby('Home')['std5_gols_sofridos_home_ft'].shift(1)

df['std5_gols_feitos_away_ft'] = df.groupby('Away')['FT_Goals_A'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True)
df['std5_gols_sofridos_away_ft'] = df.groupby('Away')['FT_Goals_H'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True)

df['std5_gols_feitos_away_ft'] = df.groupby('Away')['std5_gols_feitos_away_ft'].shift(1)
df['std5_gols_sofridos_away_ft'] = df.groupby('Away')['std5_gols_sofridos_away_ft'].shift(1)

# coeficiente de variação

df['cv5_feitos_home_ft'] = df['std5_gols_feitos_home_ft'] / df['mean55_gols_feitos_home_ft']
df['cv5_sofridos_home_ft'] = df['std5_gols_sofridos_home_ft'] / df['mean55_gols_sofridos_home_ft']

df['cv5_feitos_home_ft'] = df.groupby('Home')['cv5_feitos_home_ft'].shift(1)
df['cv5_sofridos_home_ft'] = df.groupby('Home')['cv5_sofridos_home_ft'].shift(1)

df['cv5_feitos_away_ft'] = df['std5_gols_feitos_away_ft'] / df['mean55_gols_feitos_away_ft']
df['cv5_sofridos_away_ft'] = df['std5_gols_sofridos_away_ft'] / df['mean55_gols_sofridos_away_ft']

df['cv5_feitos_away_ft'] = df.groupby('Away')['cv5_feitos_away_ft'].shift(1)
df['cv5_sofridos_away_ft'] = df.groupby('Away')['cv5_sofridos_away_ft'].shift(1)


print('pronto')

pronto


In [32]:
# media de gols 10 jogos

df['mean10_gols_feitos_home_ft'] = df.groupby('Home')['FT_Goals_H'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True)
df['mean10_gols_sofridos_home_ft'] = df.groupby('Home')['FT_Goals_A'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True)

df['mean10_gols_feitos_home_ft'] = df.groupby('Home')['mean10_gols_feitos_home_ft'].shift(1)
df['mean10_gols_sofridos_home_ft'] = df.groupby('Home')['mean10_gols_sofridos_home_ft'].shift(1)

df['mean10_gols_feitos_away_ft'] = df.groupby('Away')['FT_Goals_A'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True)
df['mean10_gols_sofridos_away_ft'] = df.groupby('Away')['FT_Goals_H'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True)

df['mean10_gols_feitos_away_ft'] = df.groupby('Away')['mean10_gols_feitos_away_ft'].shift(1)
df['mean10_gols_sofridos_away_ft'] = df.groupby('Away')['mean10_gols_sofridos_away_ft'].shift(1)

# desvio padrao 10 jogos

df['std10_gols_feitos_home_ft'] = df.groupby('Home')['FT_Goals_H'].rolling(window=10, min_periods=1).std().reset_index(0,drop=True)
df['std10_gols_sofridos_home_ft'] = df.groupby('Home')['FT_Goals_A'].rolling(window=10, min_periods=1).std().reset_index(0,drop=True)

df['std10_gols_feitos_home_ft'] = df.groupby('Home')['std10_gols_feitos_home_ft'].shift(1)
df['std10_gols_sofridos_home_ft'] = df.groupby('Home')['std10_gols_sofridos_home_ft'].shift(1)

df['std10_gols_feitos_away_ft'] = df.groupby('Away')['FT_Goals_A'].rolling(window=10, min_periods=1).std().reset_index(0,drop=True)
df['std10_gols_sofridos_away_ft'] = df.groupby('Away')['FT_Goals_H'].rolling(window=10, min_periods=1).std().reset_index(0,drop=True)

df['std10_gols_feitos_away_ft'] = df.groupby('Away')['std10_gols_feitos_away_ft'].shift(1)
df['std10_gols_sofridos_away_ft'] = df.groupby('Away')['std10_gols_sofridos_away_ft'].shift(1)

# coeficiente de variação 10 jogos

df['cv10_feitos_home_ft'] = df['std10_gols_feitos_home_ft'] / df['mean10_gols_feitos_home_ft']
df['cv10_sofridos_home_ft'] = df['std10_gols_sofridos_home_ft'] / df['mean10_gols_sofridos_home_ft']

df['cv10_feitos_home_ft'] = df.groupby('Home')['cv10_feitos_home_ft'].shift(1)
df['cv10_sofridos_home_ft'] = df.groupby('Home')['cv10_sofridos_home_ft'].shift(1)

df['cv10_feitos_away_ft'] = df['std10_gols_feitos_away_ft'] / df['mean10_gols_feitos_away_ft']
df['cv10_sofridos_away_ft'] = df['std10_gols_sofridos_away_ft'] / df['mean10_gols_sofridos_away_ft']

df['cv10_feitos_away_ft'] = df.groupby('Away')['cv10_feitos_away_ft'].shift(1)
df['cv10_sofridos_away_ft'] = df.groupby('Away')['cv10_sofridos_away_ft'].shift(1)


print('pronto')


pronto


In [33]:
# percentual mercado de gols 5 jogos

df['avg5_home_05ft'] = (df.groupby('Home')['over05ft'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg5_home_05ft'] = df.groupby('Home')['avg5_home_05ft'].shift(1)

df['avg5_away_05ft'] = (df.groupby('Away')['over05ft'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg5_away_05ft'] = df.groupby('Away')['avg5_away_05ft'].shift(1)

df['avg5_home_05ht'] = (df.groupby('Home')['over05ht'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg5_home_05ht'] = df.groupby('Home')['avg5_home_05ht'].shift(1)

df['avg5_away_05ht'] = (df.groupby('Away')['over05ht'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg5_away_05ht'] = df.groupby('Away')['avg5_away_05ht'].shift(1)

df['avg5_home_15ft'] = (df.groupby('Home')['over15ft'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg5_home_15ft'] = df.groupby('Home')['avg5_home_15ft'].shift(1)

df['avg5_away_15ft'] = (df.groupby('Away')['over15ft'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg5_away_15ft'] = df.groupby('Away')['avg5_away_15ft'].shift(1)

df['avg5_home_25ft'] = (df.groupby('Home')['over25ft'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg5_home_25ft'] = df.groupby('Home')['avg5_home_25ft'].shift(1)

df['avg5_away_25ft'] = (df.groupby('Away')['over25ft'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg5_away_25ft'] = df.groupby('Away')['avg5_away_25ft'].shift(1)

df['avg5_home_35ft'] = (df.groupby('Home')['over35ft'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg5_home_35ft'] = df.groupby('Home')['avg5_home_35ft'].shift(1)

df['avg5_away_35ft'] = (df.groupby('Away')['over35ft'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg5_away_35ft'] = df.groupby('Away')['avg5_away_35ft'].shift(1)

df['avg5_home_45ft'] = (df.groupby('Home')['over45ft'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg5_home_45ft'] = df.groupby('Home')['avg5_home_45ft'].shift(1)

df['avg5_away_45ft'] = (df.groupby('Away')['over45ft'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg5_away_45ft'] = df.groupby('Away')['avg5_away_45ft'].shift(1)

df['avg5_home_btts_ft'] = (df.groupby('Home')['btts_ft'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg5_home_btts_ft'] = df.groupby('Home')['avg5_home_btts_ft'].shift(1)

df['avg5_away_btts_ft'] = (df.groupby('Away')['btts_ft'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg5_away_btts_ft'] = df.groupby('Away')['avg5_away_btts_ft'].shift(1)

# percentual mercado de gols 10 jogos

df['avg10_home_05ft'] = (df.groupby('Home')['over05ft'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg10_home_05ft'] = df.groupby('Home')['avg10_home_05ft'].shift(1)

df['avg10_away_05ft'] = (df.groupby('Away')['over05ft'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg10_away_05ft'] = df.groupby('Away')['avg10_away_05ft'].shift(1)

df['avg10_home_05ht'] = (df.groupby('Home')['over05ht'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg10_home_05ht'] = df.groupby('Home')['avg10_home_05ht'].shift(1)

df['avg10_away_05ht'] = (df.groupby('Away')['over05ht'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg10_away_05ht'] = df.groupby('Away')['avg10_away_05ht'].shift(1)

df['avg10_home_15ft'] = (df.groupby('Home')['over15ft'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg10_home_15ft'] = df.groupby('Home')['avg10_home_15ft'].shift(1)

df['avg10_away_15ft'] = (df.groupby('Away')['over15ft'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg10_away_15ft'] = df.groupby('Away')['avg10_away_15ft'].shift(1)

df['avg10_home_25ft'] = (df.groupby('Home')['over25ft'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg10_home_25ft'] = df.groupby('Home')['avg10_home_25ft'].shift(1)

df['avg10_away_25ft'] = (df.groupby('Away')['over25ft'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg10_away_25ft'] = df.groupby('Away')['avg10_away_25ft'].shift(1)

df['avg10_home_35ft'] = (df.groupby('Home')['over35ft'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg10_home_35ft'] = df.groupby('Home')['avg10_home_35ft'].shift(1)

df['avg10_away_35ft'] = (df.groupby('Away')['over35ft'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg10_away_35ft'] = df.groupby('Away')['avg10_away_35ft'].shift(1)

df['avg10_home_45ft'] = (df.groupby('Home')['over45ft'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg10_home_45ft'] = df.groupby('Home')['avg10_home_45ft'].shift(1)

df['avg10_away_45ft'] = (df.groupby('Away')['over45ft'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg10_away_45ft'] = df.groupby('Away')['avg10_away_45ft'].shift(1)

df['avg10_home_btts_ft'] = (df.groupby('Home')['btts_ft'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg10_home_btts_ft'] = df.groupby('Home')['avg10_home_btts_ft'].shift(1)

df['avg10_away_btts_ft'] = (df.groupby('Away')['btts_ft'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg10_away_btts_ft'] = df.groupby('Away')['avg10_away_btts_ft'].shift(1)


print('pronto')



pronto


In [34]:
df['coeficiente_pontos'] = round((df['Media_Ptos_H'] - df['Media_Ptos_A']), 3)
df['cv_pontos'] = round((df['DesvPad_Ptos_H'] + df['DesvPad_Ptos_A']), 3)

df['coeficiente10_gols_feitos'] = df['mean10_gols_feitos_home_ft'] - df['mean10_gols_feitos_away_ft']
df['cv10_gols_feitos'] = df['std10_gols_feitos_home_ft'] + df['std10_gols_feitos_away_ft']

df['coeficiente10_gols_sofridos'] = df['mean10_gols_sofridos_home_ft'] - df['mean10_gols_sofridos_away_ft']


df['cv_pontos_intervalos'] = pd.qcut(df.cv_pontos, q=6, labels=['1', '2', '3', '4', '5', '6'])
df.dropna(subset=['cv_pontos_intervalos'], inplace=True)
df['cv_pontos_intervalos'] = df['cv_pontos_intervalos'].astype('int64')

df['cv_valordogol_home_intervalos'] = pd.qcut(df.CV_VG_H, q=6, labels=['1', '2', '3', '4', '5', '6'])
df.dropna(subset=['cv_valordogol_home_intervalos'], inplace=True)
df['cv_valordogol_home_intervalos'] = df['cv_valordogol_home_intervalos'].astype(float)

df['cv_valordogol_away_intervalos'] = pd.qcut(df.CV_VG_A, q=6, labels=['1', '2', '3', '4', '5', '6'])
df.dropna(subset=['cv_valordogol_away_intervalos'], inplace=True)
df['cv_valordogol_away_intervalos'] = df['cv_valordogol_away_intervalos'].astype(float)

df['valordogol_home_intervalos'] = pd.qcut(df.Media_VG_H, q=6, labels=['1', '2', '3', '4', '5', '6'])
df.dropna(subset=['valordogol_home_intervalos'], inplace=True)
df['valordogol_home_intervalos'] = df['valordogol_home_intervalos'].astype('int64')

df['valordogol_away_intervalos'] = pd.qcut(df.Media_VG_A, q=6, labels=['1', '2', '3', '4', '5', '6'])
df.dropna(subset=['valordogol_away_intervalos'], inplace=True)
df['valordogol_away_intervalos'] = df['valordogol_away_intervalos'].astype('int64')

print('pronto')


pronto


In [35]:
df.to_csv('BASE_ESTATISTICA_ARKAD_03_07.csv')